# GPS
GPS Benchmark

In [1]:
import sys
import os
import random

# Add the absolute path to the src directory 
os.chdir('../src')
src_path = os.path.abspath('../src/')
sys.path.insert(0, src_path)

from setup import *

/opt/anaconda3/envs/dsc180/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# IMDB Data

In [ ]:
# load data
imdb_train_loader, imdb_test_loader = load_imdb()
imdb_dataset = imdb_train_loader.dataset
num_node_features = imdb_dataset[0].num_node_features

# input params
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
attn_kwargs = {'dropout': 0.5}
model = GPSGraph(channels=64,\
             num_node_features = num_node_features, \
                num_layers=4, attn_type='performer', \
                    attn_kwargs=attn_kwargs).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20, min_lr=0.00001)

# train test acc
for epoch in range(150):
    loss = train_gps_graph(model, imdb_train_loader, optimizer, device)
    test_acc = test_gps_graph(model, imdb_test_loader, device)
    train_acc = test_gps_graph(model, imdb_train_loader, device)
    scheduler.step(loss)

print(f'Final Loss: {loss:.4f}, Final Train Acc: {train_acc:.4f}, Final Test Acc: {test_acc:.4f}')


Final Loss: 0.4549, Final Train Acc: 0.7750, Final Test Acc: 0.6700


# ENZYME data

In [2]:
# load data
enzyme_train_loader, enzyme_test_loader = load_enzyme()
enzyme_dataset = enzyme_train_loader.dataset
num_node_features = enzyme_dataset[0].num_node_features

# input params
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
attn_kwargs = {'dropout': 0.5}
model = GPSGraph(channels=132,\
             num_node_features = num_node_features, \
                num_layers=8, attn_type='performer', \
                    attn_kwargs=attn_kwargs).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=0.01)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20, min_lr=0.0001)

# train test acc
for epoch in range(200):
    loss = train_gps_graph(model, enzyme_train_loader, optimizer, device)
    test_acc = test_gps_graph(model, enzyme_train_loader, device)
    train_acc = test_gps_graph(model, enzyme_test_loader, device)
    scheduler.step(loss)

print(f'Final Loss: {loss:.4f},  Final Train Acc: {train_acc:.4f}, Final Test Acc: {test_acc:.4f}')


Processing...
Done!


Final Loss: -853289988.2667,  Final Train Acc: 0.1917, Final Test Acc: 0.1604


# CORA

In [8]:
# Load data
cora_dataset = Planetoid(root='/tmp/Cora', name='Cora')[0]
num_node_features = cora_dataset.num_node_features
num_classes = int(cora_dataset.y.max().item()) + 1  # Assuming labels are from 0 to num_classes - 1

# init vars and params
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GPSNode(
    num_node_features=num_node_features,
    hidden_channels=132,
    num_classes=num_classes,
    num_layers=8,
).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=20, min_lr=1e-4)

# training loop, get train test acc
for epoch in range(150):
    loss = train_gps_nodes(model, cora_dataset, optimizer, device)
    train_acc, test_acc = test_gps_nodes(model, cora_dataset, device)
    scheduler.step(loss)

print(f'Final Loss: {loss:.4f}, Final Train Acc: {train_acc:.4f} , Final Test Acc: {test_acc:.4f}')


Final Loss: 0.0002, Final Train Acc: 1.0000 , Final Test Acc: 0.7370
